In [46]:
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel
from coco_dataset import COCODataset, transform_image
from copy import deepcopy
import random
import pandas as pd
from tqdm import tqdm
import sys
import os
from train_utils import chunk_data

/tmp/ipykernel_8062/1793954293.py:5: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


Define number of keypoints of new neural network

In [47]:
NO_OF_KEYPOINTS = 17

In [48]:
path_to_coco_dataset = '/media/jakub/One Touch/coco_pose/coco2017labels-pose/coco-pose'

In [49]:
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
            [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
            [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85], [255, 255, 255]]

Create backbone model

In [50]:
backbone_model_path = '../model/body_pose_model.pth'

In [51]:
backbone_model = BodyposeBackbone()

Load backbone params from file

In [52]:
backbone_params = torch.load(backbone_model_path)

In [53]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [54]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [55]:
main_model = MainModel(backbone_model, NO_OF_KEYPOINTS)

Create dataset from COCO .json file

In [56]:
train_data = COCODataset(path_to_coco_dataset, transforms=transform_image(train=True))

loading annotations into memory...
Done (t=0.20s)
creating index...
index created!


In [57]:
# Skeleton defines which joints are connected. Because of numbering from 1 I subtract it
skeleton = train_data.coco.cats[1]['skeleton']
skeleton = [[connection[0]-1, connection[1]-1] for connection in skeleton]

In [58]:
# create list with ids where human is marked
valid_ids = []

In [59]:

for id in train_data.coco.getAnnIds():
    ann = train_data.coco.anns[id]
    if 1 in ann['keypoints'] or 2 in ann['keypoints']:
        valid_ids.append(ann['image_id'])


Show example image

In [60]:
index = random.randint(0, 100)
# load image
torch_image, image, target= train_data[index]

reference_image = image.copy()
model_output_image = image.copy()
# draw keypoints
for keypoint in target:
    for point in keypoint:
        x = point[0]
        y = point[1]
        if x != 0 and y != 0:
            cv2.circle(reference_image, (x, y), 3, (0, 0, 255), -1)
cv2.imshow('Example image', reference_image)
cv2.waitKey(10_000)
cv2.destroyAllWindows()

In [61]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [62]:
device = torch.device("cpu")

In [63]:
# image as torch tensor with shape as (CxHxW)
torch_image.size()

torch.Size([3, 480, 640])

In [64]:
output = main_model(torch_image)
part_affinity_field_maps, confidence = output
part_affinity_field_maps = part_affinity_field_maps.detach().numpy()

In [65]:
part_affinity_field_maps = np.average(part_affinity_field_maps, axis=1)
part_affinity_field_maps = np.average(part_affinity_field_maps, axis=1)

In [66]:
# chunk the data
part_affinity_field_maps_chunk = chunk_data(part_affinity_field_maps, 2)

for point in part_affinity_field_maps_chunk:
    x = point[0]
    y = point[1]
    if x != 0 and y != 0:
        cv2.circle(model_output_image, (int(x), int(y)), 3, (255, 0, 0), -1)
cv2.imshow('Example image', model_output_image)
cv2.waitKey(10_000)
cv2.destroyAllWindows()

In [67]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [68]:
def train(main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()

    main_model.train()

    logits, confidence = main_model(inputs)
    # logits = logits.detach().numpy()
    logits = torch.mean(logits, axis=1)
    logits = torch.mean(logits, axis=1)
    logits = torch.FloatTensor(logits)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    # return logits
    return output.item()

In [69]:
epochs = 1

losses = []

for epoch in range(epochs):
    for id in train_data.ids:
        print(f"epoch: {epoch}, id: {id}")
        torch_image, image, targets = train_data[id]
        targets = torch.FloatTensor(targets)
        if targets.size()[0] > 1:
            for target in targets:
                target = target[:,:-1].flatten()
                output = train(main_model=main_model, loss=loss, optimizer=optimizer, inputs=torch_image, outputs=target)
                print(f"epoch: {epoch}, id: {id}, output: {output}")
        else:
            targets = targets[:,:,:-1].flatten()
            output = train(main_model=main_model, loss=loss, optimizer=optimizer, inputs=torch_image, outputs=targets)
        print(output)

epoch: 0, id: 139


/tmp/ipykernel_8062/4154793695.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  targets = torch.FloatTensor(targets)


epoch: 0, id: 139, output: 116378.9140625
epoch: 0, id: 139, output: 42944.91015625
epoch: 0, id: 139, output: 30904985600.0
epoch: 0, id: 139, output: 10456.693359375
10456.693359375
epoch: 0, id: 785
32852.34375
epoch: 0, id: 872
epoch: 0, id: 872, output: 0.0025399706792086363
epoch: 0, id: 872, output: 0.0022729518823325634
epoch: 0, id: 872, output: 0.002252732403576374
epoch: 0, id: 872, output: 0.002230654703453183
epoch: 0, id: 872, output: 0.0022090445272624493
epoch: 0, id: 872, output: 0.002188317012041807
epoch: 0, id: 872, output: 39080.07421875
epoch: 0, id: 872, output: 0.002151861786842346
epoch: 0, id: 872, output: 47868.7734375
epoch: 0, id: 872, output: 55789.01953125
epoch: 0, id: 872, output: 0.0021289095748215914
epoch: 0, id: 872, output: 27050.021484375
epoch: 0, id: 872, output: 0.002134893322363496
epoch: 0, id: 872, output: 0.0021408703178167343
0.0021408703178167343
epoch: 0, id: 885


In [ ]:
torch_target = torch.FloatTensor(target)

In [ ]:
for target in torch_target:
    print(target.size())

torch.Size([17, 3])


In [ ]:
torch_target.size()

torch.Size([1, 17, 3])